In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from PIL import Image
import plotly.graph_objects as go
from IPython.display import display, HTML
import base64
from io import BytesIO

from iris.utils.data_utils import display_product_summary
from iris.config.data_pipeline_config_manager import DataPipelineConfigManager
from iris.config.embedding_pipeline_config_manager import EmbeddingPipelineConfigManager
from iris.data_pipeline.mongodb_manager import MongoDBManager
from iris.embedding_pipeline.embedding_handler import EmbeddingHandler
from iris.embedding_pipeline.embedding_database import EmbeddingDatabase

In [ ]:
# Initialize configuration managers
data_config = DataPipelineConfigManager()
embedding_config = EmbeddingPipelineConfigManager()

shop_config = data_config.shop_configs["nikolaj_storm"]  # Select shop
mongodb_config = data_config.mongodb_config

# Create MongoDB manager
mongodb_manager = MongoDBManager(shop_config, mongodb_config)

# Initialize the EmbeddingHandler and Database
embedding_handler = EmbeddingHandler(embedding_config.clip_config)
embedding_db = EmbeddingDatabase(embedding_config.database_config, shop_config)
embedding_db.load()

In [ ]:
# Convert cursor to list since it can only be iterated once
image_dataset = list(mongodb_manager.get_collection(
    mongodb_manager.mongodb_config.image_metadata_collection
).find())

In [ ]:
# Get the query hash's index and embedding
query_hash = '439d763d4328801469ed885e0471e695'

# Find index of the query hash in the database
hash_index = embedding_db.ids.index(query_hash)
query_embedding = embedding_db.embeddings[hash_index]

# Search for nearest neighbors
results = embedding_db.search(query_embedding, k=50)
print("Nearest neighbors for query hash", query_hash, ":", results)

In [ ]:
def get_hash_source(hash_val):
    """Get source data for a hash (either image path or mask data)"""
    # First check if it's a regular image hash
    for data in image_dataset:
        if data['image_hash'] == hash_val:
            return {'type': 'image', 'image_data': data}
        # Then check if it's a mask hash
        if 'masks' in data:
            for mask in data['masks']:
                if mask['mask_hash'] == hash_val:
                    return {'type': 'mask', 'mask_data': mask, 'parent_image': data}
    return None

def get_image_html(source, size=(100, 140)):
    """Create HTML img tag for either an image path or mask data"""
    if source['type'] == 'image':
        image_data = source['image_data']
        with Image.open(image_data['local_path']) as img:
            img.thumbnail(size)
            buffered = BytesIO()
            img.save(buffered, format='JPEG', quality=70)
            img_b64 = base64.b64encode(buffered.getvalue()).decode()
            return f'<img src="data:image/jpeg;base64,{img_b64}" style="max-width:none">'
    else:  # mask
        mask_data = source['mask_data']
        parent_data = source['parent_image']
        
        # Get bounding box from original image
        with Image.open(parent_data['local_path']) as img:
            bbox = mask_data['bbox']  # [x, y, width, height]
            bbox_img = img.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
            bbox_img.thumbnail(size)
            
            buffered = BytesIO()
            bbox_img.save(buffered, format='JPEG', quality=70)
            img_b64 = base64.b64encode(buffered.getvalue()).decode()
            return f'<img src="data:image/jpeg;base64,{img_b64}" style="max-width:none">'

In [ ]:
import numpy as np
from scipy.signal import savgol_filter

def find_distance_cutoff(distances, window=7, poly=2, base_std_multiplier=2.0, min_points=5):
    """Find cutoff point where distances start deviating significantly from the local trend.

    Args:
        distances: List of distances from nearest neighbor search
        window: Window size for smoothing (must be odd)
        poly: Polynomial order for smoothing
        base_std_multiplier: How many standard deviations to use for threshold
        min_points: Minimum number of points to return

    Returns:
        index: Cutoff index
    """
    distances = np.array(distances)
    
    # First smooth the distances to reduce noise
    smoothed = savgol_filter(distances, window, poly)
    
    # Calculate rolling statistics
    rolling_mean = [np.mean(smoothed[max(0, i-window):i+1]) 
                   for i in range(len(smoothed))]
    rolling_std = [np.std(smoothed[max(0, i-window):i+1])
                  for i in range(len(smoothed))]
    
    # Get baseline statistics from first few points
    base_mean = np.mean(smoothed[:min_points])
    base_std = np.std(smoothed[:min_points])
    
    # Find where distance exceeds mean + std threshold
    threshold = base_mean + (base_std * base_std_multiplier)

    # Look for consistent deviation
    cutoff_candidates = []
    for i in range(min_points, len(smoothed)):
        # Check if point and next few points exceed threshold
        if (smoothed[i] > threshold and
            smoothed[i] > rolling_mean[i] + rolling_std[i] * base_std_multiplier):
            cutoff_candidates.append(i)
            if len(cutoff_candidates) >= 3:  # Require multiple points above threshold
                return cutoff_candidates[0]
    
    # If no clear cutoff found, return minimum valid size
    return min_points

# Get distances from your results
distances = [dist for _, dist in results]
cutoff_idx = find_distance_cutoff(distances)

# Plot results
print(f"Suggested cutoff at index {cutoff_idx} (distance: {distances[cutoff_idx]:.4f})")

fig = go.Figure()

# Original distances
fig.add_trace(go.Scatter(
    x=list(range(len(distances))),
    y=distances,
    name='Distances',
    line=dict(color='blue')
))

# Add smoothed trace
smoothed = savgol_filter(distances, 7, 2)
fig.add_trace(go.Scatter(
    x=list(range(len(smoothed))),
    y=smoothed,
    name='Smoothed',
    line=dict(color='red', dash='dash')
))

# Highlight cutoff
fig.add_vline(
    x=cutoff_idx,
    line_dash="dash",
    line_color="green",
    annotation_text=f"Cutoff at {cutoff_idx}"
)

fig.update_layout(
    title='Distance Distribution with Cutoff',
    xaxis_title='Index',
    yaxis_title='Distance',
    width=1000,
    height=500,
    showlegend=True
)

fig.show()

# Show all images, with cutoff ones in red
html = ['<div style="display: flex; flex-wrap: wrap; gap: 10px;">']

for i, (hash_val, dist) in enumerate(results):
    source = get_hash_source(hash_val)
    if source is None:
        continue

    img_html = get_image_html(source)
    # Add red text and ❌ for items after cutoff
    text_color = 'color: red;' if i >= cutoff_idx else ''
    cutoff_marker = '❌ ' if i >= cutoff_idx else ''
    item_html = f"""
    <div style='text-align: center; border: 1px solid #ddd; padding: 5px;'>
        {img_html}
        <br>
        <small style='{text_color}'>{cutoff_marker}Index: {i}</small><br>
        <small style='{text_color}'>Distance: {dist:.4f}</small><br>
        <small style='{text_color}'>Hash: {hash_val}</small>
        <small style='{text_color}'>{'(Mask)' if source['type'] == 'mask' else ''}</small>
    </div>
    """
    html.append(item_html)

html.append('</div>')
display(HTML(''.join(html)))

In [ ]:
import plotly.express as px

# Get product scores using weighted distances for all images
scores = {}
product_titles = {}  # Store product titles for reference
title_counts = {}    # Track duplicate titles

# Get product collection reference
product_collection = mongodb_manager.get_collection(mongodb_manager.mongodb_config.product_collection)

# First get the source product of the query image
query_source = get_hash_source(query_hash)
query_product_hash = query_source['parent_image']['source_product'] if query_source['type'] == 'mask' else query_source['image_data']['source_product']

# Skip first result (index 0) since it's the query image itself
for (hash_val, dist) in results[1:]:
    source = get_hash_source(hash_val)
    if source is None:
        continue

    # Get product ID from parent image
    if source['type'] == 'mask':
        product_hash = source['parent_image']['source_product']
    else:
        product_hash = source['image_data']['source_product']

    # Get product title if not already stored
    if product_hash not in product_titles:
        product = product_collection.find_one({'product_hash': product_hash})
        title = product['title'] if product else 'Unknown Product'
        # Track duplicate titles
        title_counts[title] = title_counts.get(title, 0) + 1
        # Add hash suffix for duplicates
        if title_counts[title] > 1:
            title = f"{title} ({product_hash[:6]}...)"
        product_titles[product_hash] = title

    # Use inverse distance as weight (add small epsilon to avoid division by zero)
    weight = 1.0 / (dist + 1e-6)
    scores[product_hash] = scores.get(product_hash, 0.0) + weight

# Sort products by score
sorted_products = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Create bar colors array - highlight original product
colors = ['red' if hash_val == query_product_hash else 'lightblue' 
         for hash_val, _ in sorted_products]

# Create vertical bar chart of product scores using titles
fig = go.Figure(data=[
    go.Bar(
        y=[score for _, score in sorted_products],
        x=[product_titles[hash_val][:40] + ('...' if len(product_titles[hash_val])>40 else '') for hash_val, _ in sorted_products],
        text=[f'{score:.2f}' for _, score in sorted_products],
        textposition='auto',
        marker_color=colors
    )
])

fig.update_layout(
    title='Product Relevance Scores (Red = Query Product)',
    yaxis_title='Score',
    xaxis_title='Product Title',
    width=800,
    height=500,
    showlegend=False,
    xaxis=dict(
        tickangle=45,  # Angle the labels for better readability
        showgrid=True
    )
)

fig.show()

print("Related products (in order of relevance):")
print("-" * 50)

# Display top product using plot_product_summary
if sorted_products:
    top_product_hash = sorted_products[0][0]
    print(f"Top Product: {product_titles[top_product_hash]} (Score: {sorted_products[0][1]:.2f})")
    display_product_summary(mongodb_manager, top_product_hash)

print("-" * 50)

In [ ]:
# Update MongoDB with mask point and product hash
# Get the parent image hash from query source
parent_image_hash = query_source['parent_image']['image_hash']

# We've already found the most relevant product earlier (top_product_hash)
# Get the point coordinates from the query source
point_coords = query_source['mask_data']['point_coords'][0]  # Get first point

# Prepare the update for the mask in the image document
filter_query = {
    'image_hash': parent_image_hash,
    'masks.mask_hash': query_source['mask_data']['mask_hash']
}

# Create the update data
update_data = {
    'masks.$.mask_point': {
        'x': float(point_coords[0]),
        'y': float(point_coords[1])
    },
    'masks.$.product_hash': top_product_hash
}

# Update the document
success = mongodb_manager.update_one(
    mongodb_manager.mongodb_config.image_metadata_collection,
    filter_query,
    update_data,
    upsert=False
)

print(f"Database update {'successful' if success else 'failed'}")

# Verify the update by retrieving the document
updated_doc = mongodb_manager.find_one(
    mongodb_manager.mongodb_config.image_metadata_collection,
    {'image_hash': parent_image_hash}
)

if updated_doc:
    for mask in updated_doc['masks']:
        if mask['mask_hash'] == query_source['mask_data']['mask_hash']:
            print("\nUpdated mask data:")
            print(f"Mask hash: {mask['mask_hash']}")
            print(f"Point coordinates: {mask.get('mask_point', 'Not set')}")
            print(f"Product hash: {mask.get('product_hash', 'Not set')}")
else:
    print("Could not retrieve updated document")